In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # for plotting
from scipy.stats import zscore # for standardizing data
from sklearn.impute import SimpleImputer # for imputing missing values

In [ ]:
def checkMissingData():
    df = pd.read_csv('data.csv')
    print(df.info())
    
# checkMissingData()

In [ ]:
def missingDataHandler():
    df = pd.read_csv('data.csv')
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    missingColumns = ['Daily tests', 'Cases', 'Deaths']
    for column in missingColumns:
        df[column] = imputer.fit_transform(df[column].values.reshape(-1, 1))
    print(df.info())
    
# missingDataHandler()

In [ ]:
def addNewColumns(entity):
    df = pd.read_csv('data.csv')
    df_filter = df[df.iloc[:, 0] == entity]
    selectedColumns = ['Entity', 'Date', 'Cases', 'Deaths', "Daily tests"]
    df_new = df_filter[selectedColumns]
    df_new['Positivity Rate'] = df_new['Cases'] / df_new['Daily tests']
    df_new['Mortality Rate'] = df_new['Deaths'] / df_new['Cases']
    df_new['Date'] = pd.to_datetime(df_new['Date'])
    